<a href="https://colab.research.google.com/github/GiammarcoBozzelli/DSML/blob/main/Camembert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#imoprt packages
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import nltk
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from transformers import CamembertTokenizer, CamembertModel, pipeline
import torch

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load the dataset
url = "https://raw.githubusercontent.com/GiammarcoBozzelli/DSML/main/DATA/training_data.csv"
data = pd.read_csv(url)

# Display the first few rows of the dataset
print(data.head())

# Preprocess the data: encode labels
label_encoder = LabelEncoder()
data['Label'] = label_encoder.fit_transform(data['difficulty'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['sentence'], data['Label'], test_size=0.2, random_state=42)

   id                                           sentence difficulty
0   0  Les coûts kilométriques réels peuvent diverger...         C1
1   1  Le bleu, c'est ma couleur préférée mais je n'a...         A1
2   2  Le test de niveau en français est sur le site ...         A1
3   3           Est-ce que ton mari est aussi de Boston?         A1
4   4  Dans les écoles de commerce, dans les couloirs...         B1


In [ ]:
from transformers import CamembertTokenizer, CamembertModel
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load tokenizer and model
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
model = CamembertModel.from_pretrained("camembert-base").to(device)


In [6]:
def get_embeddings(sentences):
    # Ensure sentences are in a list and filter out any non-string entries
    sentences = [s for s in sentences if isinstance(s, str)]

    # Check if the batch is empty after filtering
    if not sentences:
        return np.array([])  # Return an empty array if no valid sentences are present

    # Tokenize and encode the batch of sentences
    inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

    # Get embeddings
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract the embeddings from the last hidden state
    embeddings = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()
    return embeddings

# Batch processing of embeddings
batch_size = 32  # Adjust batch size according to your GPU memory
X_train_embeddings = np.vstack([get_embeddings(X_train[i:i + batch_size]) for i in range(0, len(X_train), batch_size) if len(X_train[i:i + batch_size]) > 0])
X_test_embeddings = np.vstack([get_embeddings(X_test[i:i + batch_size]) for i in range(0, len(X_test), batch_size) if len(X_test[i:i + batch_size]) > 0])


In [7]:
# Create and train the logistic regression model
from sklearn import svm
clf = svm.SVC(kernel='linear')
clf.fit(X_train_embeddings, y_train)

# Predict on the test set
y_pred = clf.predict(X_test_embeddings)
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.525


In [8]:
unlabelled_url = "https://raw.githubusercontent.com/GiammarcoBozzelli/DSML/main/DATA/unlabelled_test_data.csv"
unlabelled_data = pd.read_csv(unlabelled_url)

In [10]:
unlabelled_embeddings = np.vstack([get_embeddings(unlabelled_data['sentence'][i:i + batch_size]) for i in range(0, len(unlabelled_data), batch_size) if len(unlabelled_data['sentence'][i:i + batch_size]) > 0])

In [12]:
predicted_labels = clf.predict(unlabelled_embeddings)

# Decode the predicted labels back to original labels
decoded_labels = label_encoder.inverse_transform(predicted_labels)

In [17]:
submission = pd.DataFrame({'id': unlabelled_data['id'], 'difficulty': decoded_labels})

submission.to_csv('prediction1_DT.csv', index=False)